# **SaaS Company Problem Analysis and Solution** 

**Table of content :**

1. Data Understanding
2. Find Business Problem melalui Exploratory Data Analysis and Understanding
3. Recommended Solution for Problems
****

# **Data Understanding**

### **About the Dataset**
Dataset `SaaS Sales` ini terkait dengan transaksi penjualan product-product software dari sebuah SaaS Company (B2B). Dataset ini terdiri dari 9994 baris dan 19 kolom, dengan rentang waktu 2020 hingga 2023. 

### **Data Attribute**

Tujuan utama dalam analisis ini adalah menemukan masalah bisnis yang muncul dari berbagai sisi dan menemukan aksi untuk optimalisasi profit.

**Profil Customer**

Berdasarkan data yang kita terima, kita memiliki sejumlah 99 customers dalam rentang waktu 2020 hingga 2023.

| Feature | Tipe Data | Deskripsi |
| --- | --- | --- |
| Contact Name | string | Nama Customer yang menaruh pesanan |
| Country | String | Negara asal pesanan dibuat |
| City | String | Kota asal pesanan dibuat |
| Region | String | Region asal pesanan dibuat |
| Subregion | String | Subregion asal pesanan dibuat |
| Customer | String | Nama perusahaan pemesan |
| Customer ID | Integer | Id unique untuk setiap customer |
| Industry | String | Industri perusahaan beroperasi |
| Segment | String | Segement perusahaan (SMB, Strategic, Enterprise) |

**Profil Transaksi**

Profil Transaksi memuat informasi yang menghubungkan Customer dan Pesanan mereka, sehingga dapat membantu operasional mereka berjalan.

| Feature | Tipe Data | Deskripsi |
| --- | --- | --- |
| Row ID | Integer | Id unique untuk setiap transaksi |
| Order ID | String | Id unique untuk setiap order'pesanan |
| Order Date | Date | Tanggal pemesanan dibuat |
| Date Key | Integer | Bentuk numeric dari Tanggal pemesanan |
| Product | String | Nama product dipesan |
| License | String | Lisensi key untuk product |
| Quantity | Integer | Jumlah lisensi yang dibeli untuk sebuah product |
| Sales | Integer | Jumlah pendapatan untuk setiap transaksi |
| Discount | Float | Discount untuk setiap transaksi |
| Profit | Float | Profit dari setiap transaksi |

**Scale of Measurement**

Kita perlu memahami tipe data dari setiap kolom. Di bawah ini adalah tabel yang menunjukkan tipe data bersama dengan kolom yang sesuai:

| Tipe | Feature |
| --- | --- |
| Discrete  | Row ID, Date Key, Customer ID, Quantity |
| Continuous| Sales, Discount, Profit |
| Categorical   | Order ID, Contact Name, Country, City, Region, Subregion, Customer, Industry, Segment, Product, License| 

### Import library

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [33]:
print(pd.__version__)

1.5.3


### **Data Size: Amount & Range**

Pemahaman data selanjutnya dengan melihat jumlah dan rentang dari semua atribute baik kuantitatif & kualitatif di semua variabel.

### Dataset inspection

In [3]:
# Load dataset dan convert neede data type to 'date_time'
df = pd.read_csv('SaaS-Sales.csv',parse_dates=['Order Date'])

display(df.describe(), df.describe(include=['object']))

,Row ID,Date Key,Customer ID,Sales,Quantity,Discount,Profit
count,9994.000000,9.994000e+03,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,2.021803e+07,1049.771963,229.858001,3.789574,0.156203,28.656896
std,2885.163629,1.123898e+04,29.719388,623.245101,2.225110,0.206452,234.260108
min,1.000000,2.020010e+07,1001.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,2.021052e+07,1024.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,2.022063e+07,1049.000000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,2.023052e+07,1076.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,2.023123e+07,1101.000000,22638.480000,14.000000,0.800000,8399.976000


,Order ID,Contact Name,Country,City,Region,Subregion,Customer,Industry,Segment,Product,License
count,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994
unique,5009,793,48,262,3,12,99,10,3,14,9994
top,EMEA-2023-100111,Leonard Kelly,United States,London,EMEA,NAMER,Allianz,Finance,SMB,ContactMatcher,16GRM07R1K
freq,14,37,2001,922,4219,2507,192,2127,5191,1842,1


Jumlah transaksi yang terjadi dalam tahun 2020 hingga 2023 sebanyak 9994 transaksi, dengan rincian sebagai berikut :
* Sales : Pendapatan terendah pada 0.45 dollar dan tertinggi pada 22638.48 dollar
* Quantity : Jumlah lisensi yang dibeli terendah pada 1 lisensi dan tertinggi pada 14 lisensi
* Profit : Profit terendah adalah -6599.97 dollar(Merugi) dan tertinggi pada 8399.98 dollar
* Negara pesanan dibuat : 20% pesanan dibuat dari Negara United States.
* Kota pesanan dibuat : 9.22 % pesanan dibuat dari kota London
* Region pesanan dibuat : 42.21% pesanan dari region EMEA (European, Middle East, Africa)
* Subregion : 25.08% pesanan dari subregion NAMER
* Nama Perusahaan pemesan :  1.92% pesanan datang dari perusahaan Allianz
* Industry : Industry Finance menjadi yang terbanyak dalam membeli product kita dengan proporsi 21.28%
* Segment : 51.94%, segment SMB menjadi yang terbanyak dalam membeli product SaaS kita.
* Product : Proporsi 18.43%, ContactMatcher menjadi yang terbanyak product dibeli 


### **Data Condition: Quality and Consistency**

Kita akan melakukan pengecekan untuk mengetahui masalah apa yang belum ditemukan dan belum terselesaikan terkait kualitas data. Hal tersebut untuk mengindikasikan langkah apa yang diperlukan jika terdapat masalah dalam hal kualitas serta memastikan reliability dari hasil analisis kita.

In [4]:
# Cek duplicate records
df.duplicated().sum()

0

In [5]:
listItem = []
for col in df.columns :
    listItem.append([col, df[col].isna().sum(), df[col].nunique(), df[col].unique()])

pd.DataFrame(columns=['Column Name', 'Number of Missing Value', 'Number of Unique', 'Unique Sample'], data=listItem)

,Column Name,Number of Missing Value,Number of Unique,Unique Sample
0,Row ID,0,9994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,Order ID,0,5009,"[EMEA-2022-152156, AMER-2022-138688, EMEA-2021..."
2,Order Date,0,1237,"[2022-11-09T00:00:00.000000000, 2022-06-13T00:..."
3,Date Key,0,1237,"[20221109, 20220613, 20211011, 20200609, 20230..."
4,Contact Name,0,793,"[Nathan Bell, Deirdre Bailey, Zoe Hodges, Thom..."
5,Country,0,48,"[Ireland, United States, Germany, Sweden, Cana..."
6,City,0,262,"[Dublin, New York City, Stuttgart, Stockholm, ..."
7,Region,0,3,"[EMEA, AMER, APJ]"
8,Subregion,0,12,"[UKIR, NAMER, EU-WEST, NOR, JAPN, EU, MEA, LAT..."
9,Customer,0,99,"[Chevron, Phillips 66, Royal Dutch Shell, BNP ..."


Berdasarkan obervasi dari data di atas diperoleh informasi bahwa:

* Tidak terdapat Missing Values pada semua kolom.
* Tidak terdapat Duplicate data di dataset.
* Setiap kolom memiliki data konsistensi yang baik, sehingga tidak diperlukan adanya tindakan terkait konsistensi data.
* Kualitas data sudah baik, sehingga bisa untuk dibawa ketahap lebih lanjut yaitu analisis dari sisi bisnis.

In [6]:
# Cek dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Row ID        9994 non-null   int64         
 1   Order ID      9994 non-null   object        
 2   Order Date    9994 non-null   datetime64[ns]
 3   Date Key      9994 non-null   int64         
 4   Contact Name  9994 non-null   object        
 5   Country       9994 non-null   object        
 6   City          9994 non-null   object        
 7   Region        9994 non-null   object        
 8   Subregion     9994 non-null   object        
 9   Customer      9994 non-null   object        
 10  Customer ID   9994 non-null   int64         
 11  Industry      9994 non-null   object        
 12  Segment       9994 non-null   object        
 13  Product       9994 non-null   object        
 14  License       9994 non-null   object        
 15  Sales         9994 non-null   float64 

### Exploratory Data Analysis

Pada tahap ini kita akan melihat dan mendapatkan gambaran umum mengenai data transaksi dari bisnis SaaS. Gambaran tersebut berupa performa dari perusahaan tersebut. Hal ini dapat membantu kita dalam memahami data transaksi perusahaan melalui visualisasi yang baik.

In [7]:
# rename row id column menjadi 'Transaction ID'
df.rename(columns={'Row ID':'Transaction ID'},inplace=True)

In [8]:
df.set_index('Transaction ID',inplace=True)

In [9]:
# create new column called Year
df['Year'] = df['Order Date'].dt.year

In [10]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

### **Company performance**

Performa dari perusahaan ini dapat diindikasikan melalui beberapa area : 

1. Overall Trend of Sales and Profit (2020 to 2024).
2. Sales and Profit by Regions (2020 to 2024).
3. Sales and Profit by Industries (2020 to 2024).
4. Sales and Profit by Segments (2020 to 2024).
5. Sales and Profit by Products (2020 to 2024).

#### **Analisis Sales, Profit dan Gross Profit Margin**

In [11]:
# Aggregate sales and profit by year
sales_profit_by_year = df.groupby(df['Order Date'].dt.year)[['Sales', 'Profit']].sum().reset_index()

In [12]:
sales_profit_by_year

,Order Date,Sales,Profit
0,2020,484247.4981,49543.9741
1,2021,470532.5090,61618.6037
2,2022,608473.8300,81726.9308
3,2023,733947.0232,93507.5131


In [13]:
# Creating the Altair chart
sales_chart = alt.Chart(sales_profit_by_year).mark_line(point=True, color='#FF5733').encode(
    x=alt.X('Order Date:O', title='Year'),
    y=alt.Y('Sales:Q', title='Sales', axis=alt.Axis(ticks=False, domain=False)),
    tooltip=[alt.Tooltip('Sales:Q', title='Sales', format='.2f')]
)

profit_chart = alt.Chart(sales_profit_by_year).mark_line(point=True, color='#FFBD33').encode(
    x=alt.X('Order Date:O', title='Year'),
    y=alt.Y('Profit:Q', title='Profit', axis=alt.Axis(ticks=False, domain=False)),
    tooltip=[alt.Tooltip('Profit:Q', title='Profit', format='.2f')]
)

# Layering sales and profit charts
chart = alt.layer(sales_chart, profit_chart).resolve_scale(
    y='independent'  # Ensure separate scales for y-axes
).properties(
    title='Positive Trend Sales and Profit 2020-2024',
    width=600,
    height=400
)

chart

alt.LayerChart(...)

**CAGR Formula :**

$$CAGR = (\frac{Nilai Akhir}{Nilai Awal})^\frac{1}{n}-1$$

**where**: 

- $Nilai Akhir$ : Nilai pada tahun terakhir periode.
- $Nilai Awal$ : Nilai pada tahun awal periode. 
- $n$ : Jumlah tahun

**Insights** : Dari grafik Trend Sales and Profit diatas, kita dapat menggunakan indikator (CAGR) [https://www.wallstreetprep.com/knowledge/cagr-compound-annual-growth-rate/] Compunded Annual Growth Rate untuk mengetahui kinerja perusahaan selama periode 4 tahun (2020 hingga 2023), baik untuk Sales dan Profit, sebagai berikut:

**Sales** :
Nilai Sales pada 2023 adalah 733947,02 dan Nilai Sales pada 2020 adalah 484247,50 maka nilai CAGR atau Pertumbuhan Sales Tahunan adalah : **14.86%**

**Profit** :
Nilai Profit pada 2023 adalah 93507,51 dan Nilai Profit pada 2020 adalah 49543,97 maka nilai CAGR atau Pertumbuhan Profit Tahunan adalah : **23.58%**

**Penggunaan CAGR**:
Hasil CAGR untuk sales dan profit mengindikasikan kinerja yang `baik atau positif` mengingat perusahaan SaaS dengan transaksi dan customer seperti ini termasuk ketegori `large-cap companies` [https://scripbox.com/mf/cagr-compound-annual-growth-rate/#:~:text=Size%20of%20the%20company%20and,between%20100%25%20to%20500%25.]. Perlu dipahami CAGR tidak merefleksikan volatilitas sehingga perlu disesuaikan penggunaan nya. Pada kasus/data kita tidak terdapat lonjakan atau penurunan yang ekstreme sehingga CAGR disini dapat digunakan dengan baik untuk mengukur kinerja perusahaan kita.

In [14]:
# Calculate the margin as a percentage
sales_profit_by_year['Margin'] = (sales_profit_by_year['Profit'] / sales_profit_by_year['Sales']) * 100

In [15]:
sales_profit_by_year

,Order Date,Sales,Profit,Margin
0,2020,484247.4981,49543.9741,10.231126
1,2021,470532.5090,61618.6037,13.095504
2,2022,608473.8300,81726.9308,13.431462
3,2023,733947.0232,93507.5131,12.740363


In [16]:
# Creating the Altair chart for sales and profit
chart = alt.Chart(sales_profit_by_year).mark_bar().encode(
    x=alt.X('Order Date:O', title='Year'),
    y=alt.Y('Sales:Q', title='Amount', stack=None),
    color=alt.value('#FF5733'),  # Color for sales
    tooltip=[alt.Tooltip('Sales:Q', title='Sales', format='.2f')]
).properties(
    title='Sales and Profit During 2020 - 2023',
    width=600,
    height=400
)

profit_chart = alt.Chart(sales_profit_by_year).mark_bar().encode(
    x=alt.X('Order Date:O', title='Year'),
    y=alt.Y('Profit:Q', title='Amount', stack=None),
    color=alt.value('#FFBD33'),  # Color for profit
    tooltip=[alt.Tooltip('Profit:Q', title='Profit', format='.2f')]
)

# Creating the Altair chart for margin
margin_chart = alt.Chart(sales_profit_by_year).mark_bar().encode(
    x=alt.X('Order Date:O', title='Year'),
    y=alt.Y('Margin:Q', title='Margin (%)'),
    color=alt.value('#6A5ACD'),  # Color for margin
    tooltip=[alt.Tooltip('Margin:Q', title='Margin', format='.2f')]
).properties(
    title='Gross Profit Margin (%)',
    width=600,
    height=200
)

(chart + profit_chart) & margin_chart

alt.VConcatChart(...)

**Insights** : Karena Profit berasal dari setiap transaksi maka kita dapat asumsikan margin sebagai **Gross Profit Margin**, karena tidak tersedianya data OPEX, Pajak dan Bunga untuk mendapatkan Net Profit Margin. Gross Profit Margin sebagai indikator juga dapat membantu kita untuk mengetahui profitabilitas perusahaan.

Gross Profit Margin : Gross Profit Margin CAGR = 7.58% dengan nilai awal 10.23 dan nilai akhir 12.74 , sehingga dalam periode ini perusahaan memiliki `performa yang baik` dari segi Gross Profit Margin, mengingat dengan nilai dan customer di data transaksi, perusahaan termasuk `large-cap companies`. Perusahaan lain juga melakukan perhitungan Gross Profit Margin CAGR untuk mengetahui profitabilitas dibanding dengan kompetitor yang lain [https://finbox.com/NASDAQGS:AAPL/explorer/gp_margin_cagr_3y/].

Berdasarkan [https://www.brex.com/journal/what-is-a-good-profit-margin], bisnis seperti software-as-a-service umumnya memiliki **high gross margin** karena lebih sedikit untuk Operating Costnya dan tidak memerlukan inventory jika dibanding dengan Perusahaan sejenis Grocery Stores. Walaupun tidak ada angka yang pasti namun secara umum profit margin yang baik ada diangka **5% hingga 10%**, sehingga dengan nilai CAGR 12.74% dapat dikatakan kinerja perusahaan dalam periode ini masih baik. [https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/margin.html]

#### **Analisis Sales vs Profit by Year and Region**

In [17]:
# Aggregate sales and profit by year and region
sales_profit_by_year_region = df.groupby(['Year', 'Region'])[['Sales', 'Profit']].sum().reset_index()

# Create the Altair chart for sales
sales_chart = alt.Chart(sales_profit_by_year_region).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Sales:Q', title='Sales', axis=alt.Axis(titleColor='#FF5733')),
    color=alt.value('#FF5733'),  # Color for sales
    column=alt.Column('Year:O', title='Year'),
    tooltip=[alt.Tooltip('Sales:Q', title='Sales', format='.2f')]
).properties(
    title='Sales by Region Over Time',
    width=100,  # Adjust width of each column if needed
    height=400
)

# Create the Altair chart for profit
profit_chart = alt.Chart(sales_profit_by_year_region).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Profit:Q', title='Profit', axis=alt.Axis(titleColor='#FFBD33')),
    color=alt.value('#FFBD33'),  # Color for profit
    column=alt.Column('Year:O', title='Year'),
    tooltip=[alt.Tooltip('Profit:Q', title='Profit', format='.2f')]
).properties(
    title='Profit by Region Over Time',
    width=100,  
    height=400
)

# Layer the charts with dual axes
chart = (sales_chart & profit_chart).resolve_scale(
    y='independent'  # Use independent y-axes
)

chart

alt.VConcatChart(...)

**Insights** : Analisa Sales dan Profit melalui pembagian Region memperlihatkan "Area of Interest" sebagai fokus masalah yang dapat muncul, yaitu Region APJ yang memperlihatkan profitabilitas yang buruk, ditinjukkan dengan nilai profit yang mengalami penurunan dan pernah menyebabkan kerugian pada tahun 2020 kurang lebih `-$5000`.

**Sales vs Profit by Industries per Regions**

In [18]:
sales_profit_by_industries_per_regions = df.groupby(['Region','Industry'])[['Sales','Profit']].sum().reset_index()

In [19]:
sales_profit_by_industries_per_regions

,Region,Industry,Sales,Profit
0,AMER,Communications,43019.1510,4506.8457
1,AMER,Consumer Products,85903.9650,16937.5458
2,AMER,Energy,105636.8590,18616.2875
3,AMER,Finance,173778.5990,17047.7947
4,AMER,Healthcare,107004.2910,18012.6900
5,AMER,Manufacturing,109155.6580,16487.2351
6,AMER,Misc,5644.0060,1001.2454
7,AMER,Retail,83694.5565,13363.6390
8,AMER,Tech,100253.3880,17702.7492
9,AMER,Transportation,23759.1310,3750.9574


In [20]:
# Combine sales and profit data into a single DataFrame
combined_data = pd.concat([sales_profit_by_industries_per_regions.assign(Type='Sales'),
                           sales_profit_by_industries_per_regions.assign(Type='Profit')])


chart = alt.Chart(combined_data).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Amount:Q', title='Amount'),
    color=alt.Color('Type:N', legend=alt.Legend(title="Type"), scale=alt.Scale(range=['#FF5733', '#FFBD33'])),
    column=alt.Column('Industry:N', title='Industry'),
    tooltip=[alt.Tooltip('Amount:Q', title='Amount', format='.2f')]
).transform_calculate(
    Amount='datum.Type == "Sales" ? datum.Sales : datum.Profit'  # Conditional to select sales or profit value
).properties(
    title='Sales and Profit by Industry and Region',
    width=100, 
    height=400
)

chart.display()

alt.Chart(...)

**Insights** : Region "APJ" dilihat dari Industry secara konsisten perform poorly dalam hal profitabilitas dibanding Region lain. Hal ini bisa mengindikasikan bahwa adanya strategi dari SaaS company untuk meningkatkan penetrasi product ke market APJ. 

**Visualisasi Sales dan Profit berdasarkan Segments per Region** 

In [21]:
sales_profit_by_segments_regions = df.groupby(['Region','Segment'])[['Sales','Profit']].sum().reset_index()

In [22]:
sales_profit_by_segments_regions

,Region,Segment,Sales,Profit
0,AMER,Enterprise,145193.0110,23078.0015
1,AMER,SMB,423965.9390,64474.8817
2,AMER,Strategic,268690.6545,39874.1066
3,APJ,Enterprise,66875.1460,4210.1895
4,APJ,SMB,210202.6190,-5314.4283
5,APJ,Strategic,138386.4758,12618.2257
6,EMEA,Enterprise,217584.9915,33010.4875
7,EMEA,SMB,527232.7870,74958.7558
8,EMEA,Strategic,299069.2365,39486.8017


In [23]:
# Combine sales and profit data into a single DataFrame
combined_data = pd.concat([sales_profit_by_segments_regions.assign(Type='Sales'),
                           sales_profit_by_segments_regions.assign(Type='Profit')])

chart = alt.Chart(combined_data).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Amount:Q', title='Amount'),
    color=alt.Color('Type:N', legend=alt.Legend(title="Type"), scale=alt.Scale(range=['#FF5733', '#FFBD33'])),
    column=alt.Column('Segment:N', title='Segment'),
    tooltip=[alt.Tooltip('Amount:Q', title='Amount', format='.2f')]
).transform_calculate(
    Amount='datum.Type == "Sales" ? datum.Sales : datum.Profit'  # Conditional to select sales or profit value
).properties(
    title='Sales and Profit by Segments per Region',
    width=100, 
    height=400
)

chart.display()

alt.Chart(...)

**Insights** : Untuk setiap segments company (Enterprise, SMB dan Strategic), region "APJ" memiliki performa terendah. Hal ini dibuktikan dengan profit yang rendah hingga mengalami kerugian (untuk segment SMB), hal ini memperkuat alasan untuk mengatasi masalah dari kerugian di region APJ.

In [24]:
# Aggregate the data to count the number of transactions for each segment within each region
transactions_by_segment_region = df.groupby(['Region', 'Segment']).size().reset_index(name='Transaction Count')

In [25]:
chart = alt.Chart(transactions_by_segment_region).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Transaction Count:Q', title='Number of Transactions'),
    color=alt.Color('Segment:N', legend=alt.Legend(title="Segment")),
    column=alt.Column('Segment:N', title='Segment'),
    tooltip=['Region:N', 'Segment:N', 'Transaction Count:Q']  # Show region, segment, and transaction count in the tooltip
).properties(
    title='Number of Transactions by Segment and Region',
    width=100,  # Adjust width of each column if needed
    height=400
)

chart


alt.Chart(...)

**Insights** : Sekarang kita lihat dari sisi jumlah transaksi. Secara jumlah transaksi, region "APJ" terlihat pada posisi terendah, namun dari sini belum bisa menginformasikan mengenai apakah menguntungkan atau tidak di sebuah region.

**Visualisasi Sales dan Profit Performance by Product**

In [26]:
sales_profit_by_products = df.groupby('Product')[['Sales','Profit']].sum().reset_index()

In [27]:
sales_profit_by_products

,Product,Sales,Profit
0,Alchemy,149528.0300,55617.8249
1,Big Ol Database,189238.6310,3384.7569
2,ChatBot Plugin,27118.7920,6527.7870
3,ContactMatcher,410378.2650,12496.2822
4,Data Smasher,167380.3180,41936.6357
5,FinanceHub,340935.4150,32136.4203
6,Marketing Suite,114879.9963,-3472.5560
7,Marketing Suite - Gold,223843.6080,21278.8264
8,OneView,107532.1610,18138.0054
9,SaaS Connector Pack,91705.1640,13059.1436


In [28]:
# Combine sales and profit data into a single DataFrame
combined_data = pd.melt(sales_profit_by_products, id_vars='Product', var_name='Type', value_name='Amount')


In [29]:
chart = alt.Chart(combined_data).mark_bar().encode(
    x=alt.X('Product:N', title='Product'),
    y=alt.Y('Amount:Q', title='Amount'),
    color=alt.Color('Type:N', legend=alt.Legend(title="Type"), scale=alt.Scale(range=['#FF5733', '#FFBD33'])),
    tooltip=['Product:N', 'Amount:Q']
).properties(
    title='Sales and Profit by Product',
    width=alt.Step(40),
    height=400
)

chart

alt.Chart(...)

**Insights** : Dari sisi products tidak ada pola khusus yang dapat menjadi "Turning Point" untuk meningkatkan penjualan secara siginifikan, walaupun product "Marketing Suite" terjadi profit loss namun perlu kita ingat, jika product dapat terjadi transaksi disemua region sehingga pendekatan yang akan dilakukan akan secara Region, yaitu Melakukan Pricing strategy untuk menyesuaikan kondisi market di region.

**Visualisasi Hubungan antara Profit dan Discount**

In [30]:
scatter_plot = alt.Chart(df).mark_circle(
    opacity=0.6,  
    color='#1f77b4',  
    size=60,  
).encode(
    x=alt.X('Discount:Q', title='Discount', axis=alt.Axis(grid=True)),  
    y=alt.Y('Profit:Q', title='Profit', axis=alt.Axis(grid=True)),  
    tooltip=['Discount:Q', 'Profit:Q']  
).properties(
    title='Relationship between Profit and Discount'
)

scatter_plot

alt.Chart(...)

**Insights** : Jika melihat semua transaksi yang terjadi, dapat kita lihat bahwa pemberian discount yang semakin tinggi membuat penurunan profit. Hal ini menjadi faktor utama dalam hal profitabilitas dalam periode ini 2020-2023. 

**Uji Korelasi**

In [31]:
df['Profit'].corr(df['Discount'],method='spearman')

-0.5433501822306213

In [32]:
df.corr(method='pearson')

C:\Users\Fariz\AppData\Local\Temp\ipykernel_10324\1928163937.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr(method='pearson')


,Date Key,Customer ID,Sales,Quantity,Discount,Profit,Year
Date Key,1.000000,0.004108,-0.009799,-0.005097,-0.002767,0.004609,0.999573
Customer ID,0.004108,1.000000,0.003729,-0.006892,0.003068,0.006864,0.004151
Sales,-0.009799,0.003729,1.000000,0.200795,-0.028190,0.479064,-0.009800
Quantity,-0.005097,-0.006892,0.200795,1.000000,0.008623,0.066253,-0.005788
Discount,-0.002767,0.003068,-0.028190,0.008623,1.000000,-0.219487,-0.002615
Profit,0.004609,0.006864,0.479064,0.066253,-0.219487,1.000000,0.004618
Year,0.999573,0.004151,-0.009800,-0.005788,-0.002615,0.004618,1.000000


**Insights** : Diperkuat dengan uji statistik nilai discount cukup kuat dalam menggerus profit, sehingga penyesuaian dalam pemberian nilai discount menjadi sangat diperlukan guna menigkatkan profitabilitas.  

### Main Problems and Root Cause

Berdasarkan data dan analisis diatas, setidaknya ada 2 masalah utama pada bisnis SaaS perusahaan ini:
1. Pola Penurunan profit dari sisi transaksi yang terjadi.
2. Low Profit Margin atau bahkan negative profit dari product disetiap Region.

`Root Cause` dapat disebabkan oleh:
1. Poin masalah 1 dapat disebabkan oleh `Unproportionate Discount`, dimana pada sebagian lebih dari 20% transaksi menyebabkan tidak untung. Namun ini masih dapat dilihat dari sisi strategy penetrasi product ke market.
2. Point masalah 2 dapat disebabkan oleh `Pricing Issue` dan `High Operational Cost (perbedaan currency) & Inefficient resource allocation` dengan penjelasan yaitu, Pemberian Harga bisa jadi terlalu murah dan adanya Cost yang cukup besar ketika `men-deliver` product pada perusahaan di region yang ditargetkan.  

### Recommended Actions / Solutions

1. Untuk `Root Cause 1`, perlu adanya `Proper evaluation discount on each Transaction` yang meliputi informasi mengenai competitor di setiap region dan menyeimbangkan discount yang diberikan agar profitabilitas dapat tercapai serta optimal di setiap Transaksi.
2. Untuk `Root Cause 2`, perlu adanya:
    - A. `Pricing Adjustment` untuk menyesuaikan harga yang diberikan kepada customer (menaikkan harga) namun dengan pemberian benefit yang lebih baik sebagai `trade-off` nya.
    - B. `Streamline internal process dan Invest in sales training` untuk merampingkan process `delivering` product ke customer agar dapat terjadi `cost reduction` lalu invest pada `pelatihan tim sales dalam meyakinkan customer` bahwa product-product ini adalah solusi dari permasalahan yang customer hadapi.
